In [1]:
import pandas as pd
from sklearn import svm
data = pd.read_csv('../datasets/kickstarter-2018-reduced.csv')
print data

              category main_category currency      goal  launched_utc  \
0               Poetry    Publishing      GBP    1000.0    1439295148   
1       Narrative Film  Film & Video      USD   30000.0    1504327437   
2       Narrative Film  Film & Video      USD   45000.0    1357950050   
3                Music         Music      USD    5000.0    1331954651   
4         Film & Video  Film & Video      USD   19500.0    1435998903   
5          Restaurants          Food      USD   50000.0    1456493907   
6                 Food          Food      USD    1000.0    1417458644   
7               Drinks          Food      USD   25000.0    1454357112   
8       Product Design        Design      USD  125000.0    1398363283   
9          Documentary  Film & Video      USD   65000.0    1405115748   
10          Nonfiction    Publishing      CAD    2500.0    1378750777   
11          Indie Rock         Music      USD   12500.0    1362811378   
12              Crafts        Crafts      USD    50

In [2]:
data['main_category'], main = pd.factorize(data['main_category'])
data['country'], country = pd.factorize(data['country'])
data['state'], state = pd.factorize(data['state'])
data['category'], cat = pd.factorize(data['category'])
data = data.fillna(0)
print data

       category  main_category currency      goal  launched_utc  state  \
0             0              0      GBP    1000.0    1439295148      0   
1             1              1      USD   30000.0    1504327437      0   
2             1              1      USD   45000.0    1357950050      0   
3             2              2      USD    5000.0    1331954651      0   
4             3              1      USD   19500.0    1435998903      1   
5             4              3      USD   50000.0    1456493907      2   
6             5              3      USD    1000.0    1417458644      2   
7             6              3      USD   25000.0    1454357112      0   
8             7              4      USD  125000.0    1398363283      1   
9             8              1      USD   65000.0    1405115748      1   
10            9              0      CAD    2500.0    1378750777      0   
11           10              2      USD   12500.0    1362811378      2   
12           11              5      US

In [3]:
del data['currency']
predicted = data['state']
descriptors = data
del descriptors['state']
print descriptors

       category  main_category      goal  launched_utc  backers  country  \
0             0              0    1000.0    1439295148        0        0   
1             1              1   30000.0    1504327437       15        1   
2             1              1   45000.0    1357950050        3        1   
3             2              2    5000.0    1331954651        1        1   
4             3              1   19500.0    1435998903       14        1   
5             4              3   50000.0    1456493907      224        1   
6             5              3    1000.0    1417458644       16        1   
7             6              3   25000.0    1454357112       40        1   
8             7              4  125000.0    1398363283       58        1   
9             8              1   65000.0    1405115748       43        1   
10            9              0    2500.0    1378750777        0        2   
11           10              2   12500.0    1362811378      100        1   
12          

In [4]:
# import numpy as np
# # print np.isfinite(descriptors['category']).all()
# # print np.isfinite(descriptors['main_category']).all()
# # print np.isfinite(descriptors['goal']).all()
# # print np.isfinite(descriptors['launched_utc']).all()
# # print np.isfinite(descriptors['backers']).all()
# # print np.isfinite(descriptors['country']).all()
# print np.isfinite(descriptors['usd pledged']).all()
# x = np.isfinite(descriptors['usd pledged'])
# print np.where(x == False)
# descriptors['usd pledged'][169]
# # print np.isfinite(descriptors['time_open']).all()

In [5]:
clf = svm.SVC(decision_function_shape='ovo')
clf.fit(descriptors, predicted)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [6]:
dec = clf.decision_function(descriptors[:1])
dec.shape[1]

15

In [7]:
clf.decision_function_shape = "ovr"
dec = clf.decision_function(descriptors[:1])
dec.shape[1]

6